In [1]:
from openai import Client
from dotenv import load_dotenv
import os   
from IPython.display import display, Markdown

In [2]:
load_dotenv()

True

In [3]:
class GenerationAgent:
    def __init__(self,client):
        self.client = client

    def generate(self, prompt):
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content":"You are an assistant tasked with generating initial responses to user prompts. Aim for clarity and relevance."},
                {"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content

class ReflectionAgent:
    def __init__(self,client):
        self.client = client

    def reflect(self, response):
        feedback = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": (
                    "You are a critical reviewer. Analyze the given response and provide exactly five critiques and improvements "
                    "in the format: '1. Critique: ... Improvement: ...' for each point."
                    )
                },
                {"role": "user", "content": response}]
        )
        return feedback.choices[0].message.content
class RegenerationAgent:
    def __init__(self,client):
        self.client = client
        
    def regenerate(self, query, response, feedback):
        response = self.client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an assistant tasked with refining responses. Use the original prompt, initial response, and feedback to generate an improved version."},
                {"role":"user","content":query},
                {"role": "assistant", "content": response},
                {"role": "user", "content": feedback}]
        )
        return response.choices[0].message.content
class Model:
    def __init__(self, client, max_iterations=3):
        self.client = client
        self.generation_agent = GenerationAgent(self.client)
        self.reflection_agent = ReflectionAgent(self.client)
        self.regeneration_agent = RegenerationAgent(self.client)
        self.n = max_iterations
    def generate(self, prompt,iterations=None):
        n = iterations if iterations is not None else self.n
        response = self.generation_agent.generate(prompt)
        for i in range(n):
            print(f"Iteration {i+1}")
            feedback = self.reflection_agent.reflect(response)
            response = self.regeneration_agent.regenerate(prompt, response, feedback)
        return response

In [4]:
client = Client()
query = "Write me a python code for Merge sort algorithm"
model = Model(client)
response = model.generate(query)

Iteration 1
Iteration 2
Iteration 3


In [5]:
display(Markdown(response))

Thank you for your thoughtful critiques. I've made significant improvements to the Merge Sort implementation based on your feedback. Here's the revised code that incorporates your suggestions:

```python
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Parameters:
    arr (list): The list of elements to be sorted.

    Returns:
    list: The sorted list of elements.

    Raises:
    TypeError: If the input is not a list.

    Time Complexity: O(n log n) - The algorithm is efficient for sorting large datasets.
    Space Complexity: O(n) - Additional space is used for temporary arrays during the merge process.

    This function allows any data type that supports comparison (i.e., implements the necessary 
    comparison methods). It can handle lists of integers, floats, strings, and user-defined 
    objects that support comparison. If non-comparable elements are present, a TypeError will be raised.
    """
    # Input validation
    if not isinstance(arr, list):
        raise TypeError("Input must be a list.")

    # Base case: if the array is empty or has one element, it's already sorted
    if len(arr) <= 1:
        return arr

    # Check if the list is already sorted
    if all(arr[i] <= arr[i + 1] for i in range(len(arr) - 1)):
        return arr

    mid = len(arr) // 2  # Finding the mid of the array
    left_half = merge_sort(arr[:mid])  # Sorting the left half
    right_half = merge_sort(arr[mid:])  # Sorting the right half

    # Merging the sorted left and right halves back into the original array
    return merge(left_half, right_half)

def merge(left, right):
    """
    Merges two sorted lists into a single sorted list.

    Parameters:
    left (list): The first sorted list.
    right (list): The second sorted list.

    Returns:
    list: The merged sorted list.
    """
    merged = []
    i = j = 0

    # Merging the sorted left and right halves
    while i < len(left) and j < len(right):
        if left[i] < right[j]:
            merged.append(left[i])
            i += 1
        else:
            merged.append(right[j])
            j += 1

    # Checking if any elements were left in the left_half
    while i < len(left):
        merged.append(left[i])
        i += 1

    # Checking if any elements were left in the right_half
    while j < len(right):
        merged.append(right[j])
        j += 1

    return merged

# Example usage
if __name__ == "__main__":
    # Various test cases with expected outputs
    test_cases = {
        "Unsorted array": ([38, 27, 43, 3, 9, 82, 10], [3, 9, 10, 27, 38, 43, 82]),
        "Array with duplicates": ([1, 2, 2, 2, 3, 3, 3], [1, 2, 2, 2, 3, 3, 3]),
        "Empty array": ([], []),
        "Array with one element": ([42], [42]),
        "Reverse sorted array": ([5, 4, 3, 2, 1], [1, 2, 3, 4, 5]),
        "Random order": ([1, 3, 2, 5, 4], [1, 2, 3, 4, 5]),
        "Array of strings": (["banana", "apple", "cherry"], ["apple", "banana", "cherry"]),
        "Array of lists": ([[3], [1], [2]], [[1], [2], [3]])  # Nested lists for comparability
    }

    for description, (arr, expected) in test_cases.items():
        sorted_arr = merge_sort(arr)
        print(f"Original array ({description}): {arr}")
        print(f"Sorted array: {sorted_arr}")
        # Check against expected output
        if sorted_arr == expected:
            print("Test passed!")
        else:
            print(f"Test failed! Expected: {expected}, but got: {sorted_arr}")
        print()
```

### Key Improvements:

1. **More Flexible Input Validation**: The input validation has been simplified to no longer restrict data types, allowing any type that implements comparison methods (like custom objects). The function now only checks if the input is a list.

2. **Updated Test Cases**: The tuple test case has been replaced with a case that includes nested lists, ensuring all examples contain comparable elements.

3. **Clarified Time and Space Complexity**: The docstring now explains what time and space complexity mean, helping users understand the implications of performance and memory usage.

4. **Detection of Already Sorted Input**: A new check has been added to detect if the input list is already sorted. If it is, the function returns the list immediately, avoiding unnecessary recursive calls.

5. **Validation of Results**: Each test case now checks the output against the expected sorted result, providing feedback on whether the test passed or failed with detailed output.

These enhancements make the Merge Sort implementation more robust, versatile, and user-friendly. Thank you for your guidance in refining this solution!